In [54]:
from numpy import pi
from scipy.optimize import minimize

from DIPModule import *

x0 = [0,0,0]; # init parameters
N = 1; # number of qubits

# init Hamiltonian
H = [];

# -0.5 X_i - Z_i - 2 X_i X_i+1 - 2 Z_i Z_i+1
# 'Sign' = 0 for negative, 1 for positive
for i in range(N):
    H.append(
        {'String':'I'*i+'X'+'I'*(N-i-1),'Amplitude':.5,'Sign':0}
    );
    H.append(
        {'String':'I'*i+'Z'+'I'*(N-i-1),'Amplitude':1,'Sign':0}
    );
for j in range(N-1):
    H.append(
        {'String':'I'*j+'XX'+'I'*(N-j-2),'Amplitude':2,'Sign':0}
    );
    H.append(
        {'String':'I'*j+'ZZ'+'I'*(N-j-2),'Amplitude':2,'Sign':0}
    );

# Prepare for random generation
Prob = [abs(Hi['Amplitude']) for Hi in H];
Prob = [x/sum(Prob) for x in Prob];
# Test
print(H)
print('Probability:')
print(Prob)

[{'String': 'X', 'Amplitude': 0.5, 'Sign': 0}, {'String': 'Z', 'Amplitude': 1, 'Sign': 0}]
Probability:
[0.3333333333333333, 0.6666666666666666]


# Minimization

In [55]:
# res = minimize(DIP,[0,0,0],(N,H,2000),method='nelder-mead',options={'xtol': 1e-3,'fatol': 1e-2, 'disp': True});
res = minimize(DIP,[0, 0, 0],(N,H,Prob,2000),method='Powell',options={'xtol': 1e-4,'ftol': 1e-4, 'disp': True});
res

Optimization terminated successfully.
         Current function value: -0.876010
         Iterations: 2
         Function evaluations: 86


   direc: array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])
     fun: -0.8760099007772096
 message: 'Optimization terminated successfully.'
    nfev: 86
     nit: 2
  status: 0
 success: True
       x: array([-1.16751096,  2.71045308,  0.16906383])

In [58]:
RealHamiltonian = sigmax()*H[0]['Amplitude'] + sigmaz()*H[1]['Amplitude']; # Matrix of Hamiltonian
RealState = (sigmax()+1)*Prob[0]/2 + (sigmaz()+1)*Prob[1]/2;

EigenS = RealHamiltonian.eigenstates();

trainedU = gate_sequence_product(VCirc(res['x'],1).propagators());
DiagonizedHamiltonian = trainedU*RealHamiltonian*trainedU.dag();
DiagonizedState = trainedU*RealState*trainedU.dag();

InversU = gate_sequence_product(InvVCirc(res['x'],1).propagators());
ApproxVec = InversU*StringPre([0],N)

In [59]:
print('Eigenvalues of the Hamiltonian is:')
print(EigenS[0])
print('\nEigenStates of the Hamiltonian is:')
print(EigenS[1])
print('\n\nDiagonalized Hamiltonian is:')
print(DiagonizedHamiltonian)
# print('\n\nDiagonalized State is:')
# print(DiagonizedState)
print('\nApproximated vector is:')
print(ApproxVec)

print('\n\nThe precision of the approximated ground energy is:')
print((1-np.abs(EigenS[0][0]-DiagonizedHamiltonian[0][0][0])/np.abs(EigenV[0]))*100,'%')

print('\nThe fidelity of the approximated ground state is:')
print((ApproxVec.dag()*EigenS[1][0]).norm()*100,'%')

Eigenvalues of the Hamiltonian is:
[-1.11803399  1.11803399]

EigenStates of the Hamiltonian is:
[Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[ 0.22975292]
 [-0.97324899]]
 Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[-0.97324899]
 [-0.22975292]]]


Diagonalized Hamiltonian is:
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[-1.11800374+0.j          0.00822351+0.00010238j]
 [ 0.00822351-0.00010238j  1.11800374+0.j        ]]

Approximated vector is:
Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[ 0.13238578-0.18629346j]
 [-0.5515515 +0.80222173j]]


The precision of the approximated ground energy is:
99.99752195153035 %

The fidelity of the approximated ground state is:
99.99932362211112 %
